### simple RAG using [VARAG](https://github.com/adithya-s-k/VARAG)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adithya-s-k/VARAG/blob/main/docs/simpleRAG.ipynb)

Requirement to RUN this notebook - CPU or T4(if using OCR and need fast OCR)

In [ ]:
!git clone https://github.com/adithya-s-k/VARAG
%cd VARAG
%pwd

In [ ]:
!apt-get update && apt-get install -y && apt-get install -y poppler-utils

In [ ]:
%pip install -e .

## We will be using Docling for OCR
%pip install docling

In [6]:
from sentence_transformers import SentenceTransformer
from varag.rag import SimpleRAG
from varag.llms import LiteLLM
from varag.chunking import FixedTokenChunker
import lancedb
import os
from dotenv import load_dotenv

# os.environ["OPENAI_API_KEY"] = "api-key"

load_dotenv()

INFO:datasets:PyTorch version 2.4.1 available.
INFO:httpx:HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"
/home/adithya/miniconda3/envs/vision-venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


True

In [8]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Initialize OpenAI LLM
llm = LiteLLM(model="gpt-4o-mini" , is_vision_required=True, api_key=OPENAI_API_KEY , verbose=False) 
# llm = LiteLLM(model="gpt-3.5-turbo" , is_vision_required=True, api_key=OPENAI_API_KEY) 

18:27:34 - LiteLLM:INFO: utils.py:2796 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
18:27:35 - LiteLLM:INFO: utils.py:949 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


In [10]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", trust_remote_code=True)
# embedding_model = SentenceTransformer("BAAI/bge-base-en", trust_remote_code=True)
# embedding_model = SentenceTransformer("BAAI/bge-large-en-v1.5", trust_remote_code=True)
# embedding_model = SentenceTransformer("BAAI/bge-small-en-v1.5", trust_remote_code=True)

# Initialize shared database
shared_db = lancedb.connect("./simple_rag")

# Initialize TextRAG with shared database
text_rag = SimpleRAG(
    text_embedding_model=embedding_model,
    db=shared_db,
    table_name="textDemo",
)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Initialize OpenAI LLM
llm = LiteLLM(model="gpt-4o-mini" , is_vision_required=True, api_key=OPENAI_API_KEY)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
18:27:47 - LiteLLM:INFO: utils.py:2796 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


Using device: cuda


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
18:27:47 - LiteLLM:INFO: utils.py:949 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


In [11]:
text_rag.index(
        "../examples/data/colpali.pdf",
        recursive=False,
        chunking_strategy=FixedTokenChunker(chunk_size=1000),
        metadata={"source": "gradio_upload"},
        overwrite=True,
        verbose=True,
        ocr=True,
    )

INFO:varag.rag._simpleRAG:Using OCR for file: ../examples/data/colpali.pdf
INFO:varag.rag._simpleRAG:Starting OCR conversion for file: ../examples/data/colpali.pdf
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.pipeline.base_pipeline:Processing document colpali.pdf
INFO:docling.document_converter:Finished converting document colpali.pdf in 29.88 sec.
INFO:varag.rag._simpleRAG:OCR conversion completed for file: ../examples/data/colpali.pdf
INFO:varag.rag._simpleRAG:Chunking text for file: ../examples/data/colpali.pdf
INFO:varag.rag._simpleRAG:Generated 28 chunks for file: ../examples/data/colpali.pdf
INFO:varag.rag._simpleRAG:Generating embeddings for file: ../examples/data/colpali.pdf


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:varag.rag._simpleRAG:Generated 28 embeddings for file: ../examples/data/colpali.pdf
INFO:varag.rag._simpleRAG:Adding 28 entries to the database for file: ../examples/data/colpali.pdf
INFO:varag.rag._simpleRAG:Successfully processed file: ../examples/data/colpali.pdf


'Indexing complete. Total documents in textDemo: 28'

In [12]:
query = "what is colpali ?"
num_results = 5

search_results = text_rag.search(query, k=num_results)

print("This was the retrieved Context")
for i, r in enumerate(search_results):
    print(f"{'==='*50}")
    print(f"\n\nChunk {i+1}:")
    print(f"Text: {r['text']}")
    print(f"Chunk Index: {r['chunk_index']}")
    print(f"Document Name: {r['document_name']}")
    print(f"\n\n{'==='*50}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

This was the retrieved Context


Chunk 1:
Text:  shows a table titled "System of Record" which outlines the different types of documents or records maintained across various systems or departments within an organization related to project management and construction. The rows list documents like project plans, budgets, schedules, contracts, purchase orders, invoices, change requests, bid submissions, drawings, manuals, meeting minutes, and reports. The columns indicate the system or department responsible for maintaining each record, such as County Servers, Project View, OnBase, CGI Advantage Financial System, and Purchasing Department. The table uses "W" and "T" markers to denote which system or department serves as the primary source (writer) or storage location (trailer) for each type of document.

## C More similarity maps

In Figure 7, ColPali assigns a high similarity to all patches with the word "Kazakhstan" when given the token <\_Kazakhstan> . Moreover, our model seems to exhi

In [13]:
from IPython.display import display, Markdown, Latex

context = "\n".join([r["text"] for r in search_results])
response = llm.query(
    context=context,
    system_prompt="Given the below information answer the questions",
    query=query,
)


display(Markdown(response))

18:29:00 - LiteLLM:INFO: utils.py:2796 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
18:29:02 - LiteLLM:INFO: utils.py:949 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


ColPali is a novel retrieval model architecture designed for efficient document retrieval that leverages the capabilities of Vision Language Models (VLMs). It focuses on indexing documents based purely on their visual features, allowing for quick and effective query matching through a late interaction mechanism. ColPali aims to enhance the performance of document retrieval systems by creating high-quality contextualized embeddings from images of document pages, outperforming traditional text-centric retrieval models. It was developed as part of the Visual Document Retrieval Benchmark (ViDoRe) project, which evaluates systems on page-level document retrieval tasks across various domains, emphasizing the importance of both textual and visual understanding in document retrieval applications.

### Run Gradio Demo

In [ ]:
%cd examples
!python textDemo.py --share